
Experiment Explanation

CONTEXT: We have a Question Answering service. Sometimes the service returns answers like 'Sorry, but I do not know'. Let's name these 'Sorry' answers. These answers:
* do not help users;
* can indicate that something is wrong with the context of the questions that cause these types of answers.
BUT! We use generative AI for question answering. So, it's reasonable to expect that 'Sorry' Answers will have different forms.

GOAL: To count 'Sorry' answers for a period.

HOW TO:
1. Let's gather the answers (a couple of hundreds);
2. Eliminate unique values;
3. Use an easy method to find the 'Sorry' answers.
4. Count them.

This method is a bit dummy but fast to implement.

In [1]:
import os
import psycopg2
from psycopg2 import sql

db_config = {
        'host': os.environ.get('DB_HOST'),
        'database': os.environ.get('DB_NAME'),
        'user': os.environ.get('DB_USER'),
        'password': os.environ.get('DB_PASSWORD'),
        'port': os.environ.get('DB_PORT')
    }

query = sql.SQL("select answer from messages where created_at >= timestamp with time zone '2024-02-01 00:00:00.000Z'")

connection = psycopg2.connect(**db_config)
cursor = connection.cursor()
cursor.execute(query)
result = cursor.fetchall()
cursor.close()


NameError: name 'config' is not defined

In [ ]:
import re

def formating(text):
    truncated_text = text[:50]
    pattern = r'[\x1b\t\n\r\f\v]'
    return re.sub(pattern, '', truncated_text)

answer_tup = result[0:5]
answer_list = [tup[0] for tup in result]
answer_list = [formating(item) for item in answer_list]

print(len(answer_list))

In [ ]:
from collections import Counter

# Count occurrences of each item
item_counts = Counter(answer_list)

# Filter items with count greater than 1
duplicates = {item: count for item, count in item_counts.items() if count > 1}

for item, count in duplicates.items():
    print(f"{item}: {count}")

In [ ]:
sorry_answers = [
    "orry, I can't find the answer to t",
    "orry, we don't have an answer to y",
    "orry, I don't have the informatio"
]

filtered_item_counts = {item: count for item, count in item_counts.items() if any(sub in item for sub in sorry_answers)}

# Calculate total count of all items in the original list
total_count = sum(item_counts.values())

# Calculate percentage for each item
percentage_items = {item: (count / total_count) * 100 for item, count in filtered_item_counts.items()}

for item, percentage in percentage_items.items():
    print(f"{item}: {percentage:.2f}%")